# iDigBio Differences

Development of code to construct lists of differences between two iDigBio dump parquets as made in the [GUODA data sets repo](https://github.com/bio-guoda/guoda-datasets/tree/master/iDigBio).

In [1]:
from pyspark.sql.functions import col, dayofmonth, max, month, year

In [2]:
t1_df = sqlContext.read.parquet("/guoda/data/idigbio-20180414T023309.parquet").withColumnRenamed("uuid", "t1_uuid")
t2_df = sqlContext.read.parquet("/guoda/data/idigbio-20180519T023311.parquet").withColumnRenamed("uuid", "t2_uuid")


No hope of using the existing small datasets here since the selected records are completely different every time. (Remember Nicky's suggestion that we should sort by UUID (ie random sort) and take the top ones or some other semi-determinant method that results in a good cross-section of data that is kind of stable.)

Things were going great except for the last part about add vs change and computation time got to be a drag experimenting with things. Made some smaller data sets to work with with the mk_test_data_ scripts that take in to account Nicky's suggestion.

In [3]:
# With these two test sets, adds and deletes are the same number of records.
#t1_df = sqlContext.read.parquet("/outputs/idigbio-20180414T023309-1Msorted.parquet").withColumnRenamed("uuid", "t1_uuid")
#t2_df = sqlContext.read.parquet("/outputs/idigbio-20180519T023311-1Msorted.parquet").withColumnRenamed("uuid", "t2_uuid")


In [4]:
t1_df.printSchema()

root
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dcterms:language: string (nullable = true)
 |    |-- d

## Deletes

In [5]:
deleted = (t1_df
           .join(t2_df.select(col("t2_uuid")),
                col("t1_uuid") == col("t2_uuid"), how="left")
           .filter(col("t2_uuid").isNull())
           )

In [6]:
deleted.select(col("t1_uuid"), col("t2_uuid")).orderBy(col("t1_uuid")).show(3, truncate=False)

+------------------------------------+-------+
|t1_uuid                             |t2_uuid|
+------------------------------------+-------+
|00005790-504e-47af-b8a9-6309f4a9ad52|null   |
|00058994-6235-4a37-a620-5601f78461d7|null   |
|0005e843-ffeb-4683-8ea1-2ba75f548b26|null   |
+------------------------------------+-------+
only showing top 3 rows



In [7]:
a_deleted_uuid = "00005790-504e-47af-b8a9-6309f4a9ad52"

In [8]:
print(deleted.count())

75651


Make sure we got the join in the right direction, look for a "deleted" uuid in the second data frame, should not be there.

In [11]:
(t2_df
 .filter(col("t2_uuid") == a_deleted_uuid)
 .select(col("t2_uuid"))
 .show()
)

+-------+
|t2_uuid|
+-------+
+-------+



And look for it in the first data frame, it should be there:

In [12]:
(t1_df
 .filter(col("t1_uuid") == a_deleted_uuid)
 .select(col("t1_uuid"))
 .show()
)

+--------------------+
|             t1_uuid|
+--------------------+
|00005790-504e-47a...|
+--------------------+



## Adds

In [13]:
added = (t2_df
           .join(t1_df.select(col("t1_uuid")),
                col("t1_uuid") == col("t2_uuid"), how="left")
           .filter(col("t1_uuid").isNull())
           )

In [14]:
added.printSchema()

root
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dcterms:language: string (nullable = true)
 |    |-- d

In [15]:
added.select(col("t2_uuid"), col("t1_uuid")).orderBy(col("t2_uuid")).show(3, truncate=False)

+------------------------------------+-------+
|t2_uuid                             |t1_uuid|
+------------------------------------+-------+
|0000075c-4e94-4a42-9be4-fbd32201c71f|null   |
|0000095f-2150-4d4a-aee3-2c4622474587|null   |
|00000e6c-ad9d-4476-8025-47ad4e886336|null   |
+------------------------------------+-------+
only showing top 3 rows



In [16]:
a_added_uuid = "0000075c-4e94-4a42-9be4-fbd32201c71f"

In [17]:
print(added.count())

949511


Make sure we got the join in the right direction, look for a "added" uuid in the first data frame, should not be there.

In [19]:
(t1_df
 .filter(col("t1_uuid") == a_added_uuid)
 .select(col("t1_uuid"))
 .show()
)

+-------+
|t1_uuid|
+-------+
+-------+



And look for it in the second data frame, it should be there:

In [20]:
(t2_df
 .filter(col("t2_uuid") == a_added_uuid)
 .select(col("t2_uuid"))
 .show()
)

+--------------------+
|             t2_uuid|
+--------------------+
|0000075c-4e94-4a4...|
+--------------------+



Added and deleted have the same number when we are testing with the sorted and limited data sets, regardless of which two we pick to difference. This is too much coincidence.

In [21]:
(added.select(col("t2_uuid").alias("added_uuid"))
 .join(deleted.select(col("t1_uuid").alias("deleted_uuid")),
       col("added_uuid") == col("deleted_uuid"))
 .show(10, truncate=False)
 )

+----------+------------+
|added_uuid|deleted_uuid|
+----------+------------+
+----------+------------+



But the added and deleted sets are certainly different as show above. In thinking about this more, because we've sorted and then cut the list of UUIDs at a fixed number, if we add X new records, we're also removing X records because we're capped at only 1M records total. This is fine for testing, let's see what happens when we use the whole dataset, there should be different counts.

## Updates

We'l rely on the last_mod time being correct here. Could we use this for adds too? (See more discussion of this lower.)

In [22]:
last_updated = (t1_df
               .select(max(col("datemodified")).alias("last_updated"))
               .collect()
                )[0]["last_updated"]
print(last_updated)

2018-04-06 10:47:43.476000


In [23]:
updated = (t2_df
          .filter(col("datemodified") > last_updated)
          )


In [24]:
(updated
 .select(col("t2_uuid"), col("datemodified"))
 .orderBy(col("datemodified"))
 .show(3, truncate=False)
 )

+------------------------------------+-----------------------+
|t2_uuid                             |datemodified           |
+------------------------------------+-----------------------+
|7683f3be-82d5-44a7-b506-71a6a20bbfa2|2018-04-12 10:38:54.349|
|7692b941-9baf-484e-a01a-5a544c7b482b|2018-04-12 10:38:54.377|
|a3335eca-f514-45a6-9796-89127d2c4c87|2018-04-12 10:38:54.377|
+------------------------------------+-----------------------+
only showing top 3 rows



In [25]:
print(updated.count())

20525304


How often is ingestion run? Does it make sense that the last mod and the first changed date are so far apart? Let's group by day (maybe ingestion takes less than a day?)

In [26]:
ingestions = (t2_df
.select(year(col("datemodified")).alias("year"),
       month(col("datemodified")).alias("month"),
       dayofmonth(col("datemodified")).alias("day"))
 .groupBy(col("year"), col("month"), col("day"))
 .count()
 .orderBy([col("year"), col("month"), col("day")], ascending=[False, False, False])
)

In [27]:
ingestions.show(10, truncate=False)

+----+-----+---+--------+
|year|month|day|count   |
+----+-----+---+--------+
|2018|5    |9  |5093938 |
|2018|4    |23 |1777326 |
|2018|4    |16 |2371    |
|2018|4    |13 |1457214 |
|2018|4    |12 |12194455|
|2018|4    |3  |51416   |
|2018|3    |21 |150528  |
|2018|3    |9  |475311  |
|2018|3    |2  |3246062 |
|2018|1    |26 |94031   |
+----+-----+---+--------+
only showing top 10 rows



In [36]:
print(109394 + 13159 + 23 + 16053 + 46312) # test data set summation of modified records

184941


That looks about when Dan would have been running ingestion. Do we pick up added things in the updated data frame?

In [29]:
(updated
 .filter(col("t2_uuid") == a_added_uuid)
 .select(col("t2_uuid"))
 .show(3, truncate=False)
)

+------------------------------------+
|t2_uuid                             |
+------------------------------------+
|0000075c-4e94-4a42-9be4-fbd32201c71f|
+------------------------------------+



Ok, there's one, are they all there?

In [30]:
double_added = (added.select(col("t2_uuid").alias("added_uuid"))
           .join(updated.select(col("t2_uuid").alias("updated_uuid")),
                 col("added_uuid") == col("updated_uuid"), how="left")
           .filter(col("updated_uuid").isNull())
           )

In [31]:
print(double_added.count())

0


So yup, all addeds have a matching updated.

datemodified filter does pick up all additions as well as changes. How important is it to know the difference? I want to know for stats reporting so let's track them separately anyway. Let's try this method of calculating changed instead of the above.

In [32]:
only_updated = (t2_df
                .filter((t2_df.datemodified > last_updated)) #& added.uuid.isNotNull())
          .join(added.select(col("t2_uuid").alias("added_uuid")), 
                col("t2_uuid") == col("added_uuid"), how="left")
                .filter(col("added_uuid").isNull())
          )

In [33]:
print(only_updated.count())

19575793


In [37]:
print(184941 - 8541) # test data sets update - added check

176400


Hooray! Things check out.

Careful use of alias() and select() in joins has cured the weird behavior we were seeing before.

Now have three operations for finding adds, deletes, and updates. Union these sets together with a new column that indicates which row is which kind of operation and export this as our diff.